In [6]:
import pandas as pd

df_satin = pd.read_excel("./data/2022년_공단직영_상위_충전량_10위_요약.xlsx")
df_satin

,주소,addr,addr(로드뷰로_확인전),위도(Latitude),경도(Longitude),충전량(kWh),특이사항
0,시청별관,정확한 주소 없음,연암로 60 (입구50m 우측),35.890672,128.600535,97547.18,NaN
1,농수산물도매시장,대구광역시 북구 매천동 526,매천로18길 34 (4문 좌측),35.901915,128.541006,82014.83,NaN
2,농수산물도매시장2,대구 북구 매천로18길 34,매천로18길 34 (4문 좌측),35.901794,128.541782,82014.83,NaN
3,아양아트센터,대구 동구 효동로2길 24,효동로 2길 24,35.879509,128.652736,77083.54,NaN
4,동구청,대구 동구 아양로 207 동구청 (신암동 36-1),아양로 207 (입구 우측),35.886491,128.635772,76874.81,주차장에 충전소가 있는데 나무때문에 그늘짐.
5,북구청,대구 북구 옥산로 65,옥산로 65 (별관 입구 좌측),35.884923,128.583276,68311.98,NaN
6,충혼탑주차장,대구 남구 대명동 417-52,앞산순환로 540 (입구 좌측),35.832754,128.585479,64537.74,NaN
7,달서구청,대구 달서구 학산로 45 달서구청 (월성동 281),학산로 45 (입구 좌측),35.829420,128.533027,62135.23,주차장에 충전소가 있는데 나무때문에 그늘짐.
8,대구공공시설관리공단(지산),무학로 112 (입구 좌측),무학로 112 (입구 좌측),35.829637,128.621347,60740.71,NaN
9,파동교네거리주차장,정확한 주소 없음,파동 274 (입구 우측),35.810031,128.618947,60564.27,NaN


In [7]:
import requests
import ssl
from requests.adapters import HTTPAdapter


# SSL/TLS 컨텍스트 설정 변경
class SSLAdapter(HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, *args, **kwargs):
        kwargs['ssl_context'] = self.ssl_context
        return super().init_poolmanager(*args, **kwargs)

ssl_context = ssl.create_default_context()
ssl_context.set_ciphers('DEFAULT:@SECLEVEL=1')

adapter = SSLAdapter(ssl_context=ssl_context)
session = requests.Session()
session.mount('https://', adapter)


In [32]:
import pandas as pd

# 시계열 일사량 api로 데이터를 가져오는 함수
def get_solar_api_data(latitude, longitude, date):

    # URL에 변수 삽입
    url = f"https://apis.data.go.kr/B551184/openapi/service/SolarGhiService/getSolarGhiHrInfo?serviceKey=KmJehGYlsJSeUOwlr80UIbVByWokvs7BEY0bv5sRkdAR6mrcK2TDWWf6UAbukCfhxUuCG4ktHBpriZPXYwbS0g%3D%3D&pageNo=1&numOfRows=50&date={date}&lat={latitude}&lon={longitude}&type=json"

    
    response = session.get(url)
    print(response.text)

    if response.status_code != 200:
            print(f"API 요청에 실패했습니다. 날짜: {date}, 에러: HTTP {response.status_code}")
            return None

    # JSON 응답을 파싱
    data = response.json()
    return data

# 시계열 일사량를 데이터프레임으로 만드는 함수
def solar_data_to_df(data, name):
    # 필요한 데이터 추출
    items = data['response']['body']['items']['item']

    # 데이터프레임으로 변환
    df = pd.DataFrame(items)

    # name 열 추가 및 name 변수 값 삽입
    df.insert(0, 'name', name)  # 첫 번째 열로 name 열을 삽입

    # 데이터프레임 리턴
    return df

In [34]:
from datetime import datetime, timedelta


# 2022년의 모든 날짜를 YYYYMMDD형식의 문자를 리스트로 반환하는 함수이다.
def date_2022_generation():
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2022, 12, 31)
    date_generated = [(start_date + timedelta(days=x)).strftime('%Y%m%d') for x in range(0, (end_date - start_date).days + 1)]
    return date_generated

date_2022 = date_2022_generation()

In [35]:
# 현재는 2022년 1월 1일의 데이터만 데이터 프레임으로 만들수 있지만 2022년 1월 1일부터 2022년 12월 31일까지의 데이터를 df_satin_solor[0]에 넣고싶음
data_solar = get_solar_api_data(df_satin['위도(Latitude)'][0], df_satin['경도(Longitude)'][0], date_2022[0])

{
  "response": {
    "header": {
      "resultCode": "00",
      "resultMsg": "NORMAL_SERVICE"
    },
    "body": {
      "numOfRows": "50",
      "pageNo": "1",
      "totalCount": "0",
      "items": {}
    }
  }
}


In [31]:
df_satin_solor = []

if not data_solar['response']['body']['items'] :
    print(f"{df_satin['주소'][0]} : {date_2022[0]}의 데이터는 비어있음")
else:
    df_satin_solor.append(solar_data_to_df(data_solar, df_satin['주소'][0]))


df_satin_solor

시청별관 : 20220101의 데이터는 비어있음


[]

In [36]:
def gather_solar_data_for_year(df_satin):
    date_2022 = date_2022_generation()
    df_satin_solor = pd.DataFrame()
    
    for date in date_2022:
        data_solar = get_solar_api_data(df_satin['위도(Latitude)'][0], df_satin['경도(Longitude)'][0], date)
        
        if data_solar and data_solar['response']['body']['items']:
            df_daily = solar_data_to_df(data_solar, df_satin['주소'][0])
            df_satin_solor = pd.concat([df_satin_solor, df_daily], ignore_index=True)
        else:
            print(f"{df_satin['주소'][0]} : {date}의 데이터는 비어있음")
    
    return df_satin_solor


In [39]:
def gather_solar_data_for_year(df_satin):
    date_2022 = date_2022_generation()
    df_satin_solor = pd.DataFrame()
    
    for i, date in enumerate(date_2022):
        data_solar = get_solar_api_data(df_satin['위도(Latitude)'][0], df_satin['경도(Longitude)'][0], date)
        
        if data_solar and data_solar['response']['body']['items']:
            df_daily = solar_data_to_df(data_solar, df_satin['주소'][0])
            df_satin_solor = pd.concat([df_satin_solor, df_daily], ignore_index=True)
            print(f"{i + 1}/{len(date_2022)}: {date} 데이터 처리 완료")
        else:
            print(f"{df_satin['주소'][0]} : {date}의 데이터는 비어있음")
    
    return df_satin_solor


In [41]:
df_satin_solor = gather_solar_data_for_year(df_satin)


{
  "response": {
    "header": {
      "resultCode": "00",
      "resultMsg": "NORMAL_SERVICE"
    },
    "body": {
      "numOfRows": "50",
      "pageNo": "1",
      "totalCount": "0",
      "items": {}
    }
  }
}
시청별관 : 20220101의 데이터는 비어있음
{
  "response": {
    "header": {
      "resultCode": "00",
      "resultMsg": "NORMAL_SERVICE"
    },
    "body": {
      "numOfRows": "50",
      "pageNo": "1",
      "totalCount": "26",
      "items": {
        "item": [
          {
            "Date": "2022-01-02",
            "time": "08:00",
            "lat": "35.8917331287",
            "lon": "128.6046379933",
            "ghi": "0.00000",
            "cghi": "0.00000"
          },
          {
            "Date": "2022-01-02",
            "time": "09:00",
            "lat": "35.8917331287",
            "lon": "128.6046379933",
            "ghi": "0.00000",
            "cghi": "0.00000"
          },
          {
            "Date": "2022-01-02",
            "time": "10:00",
            "l

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [42]:
df_satin_solor

[]

<h2> 다음은 개선된 함수 </h2>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# 2022년 전체 데이터를 가져와서 하나의 데이터프레임으로 합치는 코드
def get_full_year_solar_data(name, latitude, longitude):
    df_solor = pd.DataFrame()
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2022, 12, 31)
    date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date - start_date).days + 1)]

    for date in date_generated:
        date_str = date.strftime("%Y%m%d")
        data = get_solar_api_data(latitude, longitude, date_str)
        if data:
            df_daily = solar_data_to_df(data)
            if not df_daily.empty:
                # name 열 추가 및 name 변수 값 삽입
                df_daily['name'] = name
                # year, month, day 열 추가 및 값 삽입
                df_daily['year'] = date.year
                df_daily['month'] = date.month
                df_daily['day'] = date.day
                df_solor = pd.concat([df_solor, df_daily], ignore_index=True)
            else:
                print(f"빈 데이터가 반환되었습니다. 날짜: {date_str}")
        else:
            print(f"API 데이터 가져오기 실패, 날짜: {date_str}")

    # name, year, month, day 열을 데이터프레임의 앞쪽에 배치
    cols = ['name', 'year', 'month', 'day'] + [col for col in df_solor.columns if col not in ['name', 'year', 'month', 'day']]
    df_solor = df_solor[cols]

    return df_solor
